# 합성곱 신경망(CNN)

In [1]:
#라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [3]:
#데이터셋 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

#transfom.Compose([]): 리스트 안에 있는 여러 전처리 작업을 순차적으로 적용.

#transform.ToTensor(): 이미지를 PIL 이미지나 NumPy 배열 형태에서 PyTorch의 텐서 형태로 변환
# -> 이미지의 픽셀 값이 보통 [0, 255] 범위를 가지는데, 이를 [0,1] 범위로 스케일링.
# -> 예를 들어, 255는 1로, 128은 0.5로 변환

#transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# : Normalize(mean, std)는 각 채널의 평균과 표준편차를 기준으로 텐서 값을 정규화
# - (0.5, 0.5, 0.5): 각 채널의 평균을 0.5로 설정.
# - (0.5, 0.5, 0.5): 각 채널의 표준편차를 0.5로 설정합니다.
# 픽셀 값을 (x - mean) / std로 변환하여, 이미지의 각 채널을 [-1, 1] 범위로 조정합니다.
# 주요 목적: 데이터 분포를 평균 0, 표준편차 1에 가깝게 조정하여 학습을 안정화시키고 모델이 학습하는 속도를 향상시키는 데 있습니다.

In [5]:
#CIFAR-10 데이터셋 로드
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
#간단한 CNN 모델 정의
class SimpleCNN(nn.Module): #nn.module 상속받기
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3,32,3, padding=1) #입력채널: 3, 출력 채널:32, 커널 크기: 3x3, 패딩: 1로 설정하여 입력과 출력 크기를 같게 유지.
        self.pool = nn.MaxPool2d(2,2) #풀링 종류: MaxPool2d, 커널 크기: 2x2(입력 크기를 절반으로 줄임), 스트라이트:2(커널이 한 번에 두 칸씩 이동하여 특징 맵 크기를 줄임)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #입력채널: 32(첫 번째 컨볼루션 계층의 출력), 출력 채널: 64(더 많은 특징을 추출), 커널 크기: 3x3 필터, 패딩: 1(출력 크기 유지)
        self.fc1 = nn.Linear(64*8*8, 512) #64개의 8*8 특징 맵을 평탄화하여 입력으로 사용하고, 출력은 512개의 노드로 구성된 완전 연결 계층
        self.fc2 = nn.Linear(512,10) #512개의 입력을 받아 10개의 출력으로 변환

    def forward(self, x): #순전파
        x = self.pool(torch.relu(self.conv1(x))) #입력 x를 conv1 계층에 통과시키고, ReLU 활성화 함수를 적용하여 비선형성을 추가한 후 pool 계층을 통과
        x = self.pool(torch.relu(self.conv2(x))) #첫 번째 컨볼루션의 출력을 두 번째 컨볼루션 계층과 풀링 계층에 통과
        x = x.view(-1, 64*8*8) #flatten: CNN 계층의 출력을 1차원으로 변환하여 완전 연결 계층에 입력할 수 있게함. 64*8*8은 conv2와 pool 계층을 통과한 후의 텐서 크기임
        x = torch.relu(self.fc1(x)) #첫 번쨰 완전 연결 계층 fc1에 입력하여 활성화 함수를 적용하고
        x = self.fc2(x) #마지막 계층 fc2에서 최종 출력값 얻음
        return x

In [21]:
for i, data in enumerate(trainloader, 0): #enumerate(trainloader, 0)는 trainloader의 데이터 배치를 하나씩 가져오며, 첫 번째 반복에서 인덱스 i는 0부터 시작
    inputs, labels = data #data 튜플을 unpacking하여 inputs와 labels를 가져옴
    print(inputs.shape)

#출력값 해설
#torch.size([64,3,32,32])
#64: 배치 크기,
#3: 채널 크기,
#32: 가로 폭,
#32: 세로 폭

torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 3

In [22]:
#모델 초기화
model = SimpleCNN()


#손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss() #criterion:  손실함수, nn.CrossEntropyLoss()는 분류 문제에 적합한 손실 함수로, 모델의 예측값과 실제 레이블 간의 차이를 계산
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) #optimizer: 최적화 알고리즘, SGD를 사용하여, 모델의 가중치 학습 - lr=0.01: 학습률을 0.01로 설정, momentum=0.9: 관성을 적용해 기울기를 보완하여 더 빠르게 최적화될 수 있도록 함.

#모델 학습
for epoch in range(10): #10에포크 동안 학습
    running_loss = 0.0
    for i, data in enumerate(trainloader,0): #trainloader에서 미니배치를 하나씩 불러옴
        inputs, labels = data

        #기울기 초기화
        optimizer.zero_grad() #역전파 이전에 모든 가중치의 기울기를 초기화. PyTorch에서는 기울기가 누적되므로 zero_grad()를 호출하여 이전 미니배치에서 계산된 기울기를 제거

        #순전파 + 역전파 + 최적화
        outputs = model(inputs) #순전파
        loss = criterion(outputs, labels) #손실계산
        loss.backward() #역전파
        optimizer.step() #최적화

        #손실 출력
        running_loss += loss.item()
        if i % 100 == 99:
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
    
print('학습 종료')

[Epoch 1, Batch 100] loss: 2.105
[Epoch 1, Batch 200] loss: 3.857
[Epoch 1, Batch 300] loss: 5.408
[Epoch 1, Batch 400] loss: 6.862
[Epoch 1, Batch 500] loss: 8.243
[Epoch 1, Batch 600] loss: 9.586
[Epoch 1, Batch 700] loss: 10.873
[Epoch 2, Batch 100] loss: 1.137
[Epoch 2, Batch 200] loss: 2.250
[Epoch 2, Batch 300] loss: 3.346
[Epoch 2, Batch 400] loss: 4.444
[Epoch 2, Batch 500] loss: 5.454
[Epoch 2, Batch 600] loss: 6.466
[Epoch 2, Batch 700] loss: 7.454
[Epoch 3, Batch 100] loss: 0.858
[Epoch 3, Batch 200] loss: 1.729
[Epoch 3, Batch 300] loss: 2.615
[Epoch 3, Batch 400] loss: 3.495
[Epoch 3, Batch 500] loss: 4.360
[Epoch 3, Batch 600] loss: 5.218
[Epoch 3, Batch 700] loss: 6.050
[Epoch 4, Batch 100] loss: 0.705
[Epoch 4, Batch 200] loss: 1.408
[Epoch 4, Batch 300] loss: 2.131
[Epoch 4, Batch 400] loss: 2.838
[Epoch 4, Batch 500] loss: 3.539
[Epoch 4, Batch 600] loss: 4.235
[Epoch 4, Batch 700] loss: 4.921
[Epoch 5, Batch 100] loss: 0.528
[Epoch 5, Batch 200] loss: 1.069
[Epoch 5,

In [23]:
#모델 평가
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%') #.2f는 소수점 아래 두 자리까지 표시하라는 것

Accuracy of the network on the 10000 test images: 74.13%
